In [8]:
import pandas as pd
pd.options.display.float_format = '{:,.0f}'.format
# pd.options.display.int_format ='{:,}'.format

import os
from operator import itemgetter    
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
get_ipython().magic(u'matplotlib inline')
plt.style.use('ggplot')


In [9]:
import_csv_file = '../_data/operations_fulldata_nodup.csv'
df = pd.read_csv(import_csv_file)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128031 entries, 0 to 128030
Data columns (total 58 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   op_id              128031 non-null  int64  
 1   subject_id         128031 non-null  int64  
 2   hadm_id            128031 non-null  int64  
 3   opdate             128031 non-null  int64  
 4   age                128031 non-null  int64  
 5   sex                128031 non-null  object 
 6   weight             126611 non-null  float64
 7   height             127269 non-null  float64
 8   race               128031 non-null  object 
 9   asa                124636 non-null  float64
 10  emop               128031 non-null  int64  
 11  department         128031 non-null  object 
 12  antype             128031 non-null  object 
 13  icd10_pcs          128031 non-null  object 
 14  category_desc      128031 non-null  object 
 15  desc_short         128031 non-null  object 
 16  ca

In [10]:
# calculate teh operation durations and add to DF
df['op_duration']=df['orout_time']-df['orin_time']
df['LOS']=(df['discharge_time']-df['orout_time'])/1440 # LOS is time following surgery to dishcarge converted to days. 
operations_info = df.loc[:,['op_id','category_id','category_desc','desc_short', 'icd10_pcs', 'orin_time','orout_time', 'op_duration', 'LOS']]

operations_info.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128031 entries, 0 to 128030
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   op_id          128031 non-null  int64  
 1   category_id    128031 non-null  object 
 2   category_desc  128031 non-null  object 
 3   desc_short     128031 non-null  object 
 4   icd10_pcs      128031 non-null  object 
 5   orin_time      128031 non-null  int64  
 6   orout_time     128031 non-null  int64  
 7   op_duration    128031 non-null  int64  
 8   LOS            128031 non-null  float64
dtypes: float64(1), int64(4), object(4)
memory usage: 8.8+ MB


In [11]:
import pandas as pd

TOP_VAR_PROCEDURES = 100

# Group the DataFrame by 'icd10_pcs' and 'desc_schort' and calculate descriptive statistics for 'LOS'
descriptive_stats = operations_info.groupby(['category_id','category_desc'])['LOS'].describe()

# use the agg() method with a custom list of percentiles
percentiles = [0.1, 0.9]  # 
percentile_stats = operations_info.groupby(['category_id','category_desc'])['LOS'].quantile(percentiles).unstack()

# # Concatenate the percentile stats to the descriptive stats
descriptive_stats = pd.concat([descriptive_stats, percentile_stats], axis=1)

# # Rename the percentile columns for clarity
descriptive_stats = descriptive_stats.rename(columns={0.1: '10%', 0.9: '90%'})

# MODE or additional quantiles if needed
# descriptive_stats['mode'] = operations_info.groupby(['icd10_pcs','desc_short'])['LOS'].agg(pd.Series.mode)

operations_desc_stats_df = descriptive_stats.sort_values(by='std', ascending=False).head(TOP_VAR_PROCEDURES)
# descriptive_stats.to_csv('los.csv')

operations_desc_stats_df

,,count,mean,std,min,25%,50%,75%,max,10%,90%
category_id,category_desc,,,,,,,,,,
0B1,Respiratory System,335,68,253,0,11,22,55,"3,711",5,100
001,Central Nervous System and Cranial Nerves,576,48,149,1,7,11,31,"1,778",5,93
00P,Central Nervous System and Cranial Nerves,10,64,145,1,9,13,31,475,7,100
0BQ,Respiratory System,61,21,114,0,2,3,8,896,1,12
009,Central Nervous System and Cranial Nerves,"1,651",14,105,1,3,5,8,"3,728",2,22
...,...,...,...,...,...,...,...,...,...,...,...
0HR,Skin and Breast,519,10,10,0,5,7,9,72,1,18
0Q9,Lower Bones,6,9,10,2,3,3,12,28,2,21
0RH,Upper Joints,47,8,10,1,3,5,8,60,1,19


In [12]:
## Clean up the index. 

operations_desc_stats_df.reset_index(inplace=True)
operations_desc_stats_df.index
operations_desc_stats_df


,category_id,category_desc,count,mean,std,min,25%,50%,75%,max,10%,90%
0,0B1,Respiratory System,335,68,253,0,11,22,55,"3,711",5,100
1,001,Central Nervous System and Cranial Nerves,576,48,149,1,7,11,31,"1,778",5,93
2,00P,Central Nervous System and Cranial Nerves,10,64,145,1,9,13,31,475,7,100
3,0BQ,Respiratory System,61,21,114,0,2,3,8,896,1,12
4,009,Central Nervous System and Cranial Nerves,"1,651",14,105,1,3,5,8,"3,728",2,22
...,...,...,...,...,...,...,...,...,...,...,...,...
95,0HR,Skin and Breast,519,10,10,0,5,7,9,72,1,18
96,0Q9,Lower Bones,6,9,10,2,3,3,12,28,2,21
97,0RH,Upper Joints,47,8,10,1,3,5,8,60,1,19
98,0QQ,Lower Bones,8,14,10,3,7,11,17,33,5,26


In [13]:
## Export List of PCD codes with greatest STD DEV to CSV

# EXPORT and Specify the file path 
output_csv_file = '../_data/category_STD.csv'

# Save the DataFrame to a CSV file
operations_desc_stats_df.to_csv(output_csv_file, index=False)

In [14]:
## Create a list of the surgeries with the greatest Variances in Length of stay
std_devs = operations_desc_stats_df['category_id']
print(std_devs)

# # Now, filter the DataFrame to these top 100 codes
top_LOSvariance_df = operations_info[operations_info['category_id'].isin(std_devs)]

top_LOSvariance_df



0     0B1
1     001
2     00P
3     0BQ
4     009
     ... 
95    0HR
96    0Q9
97    0RH
98    0QQ
99    0D1
Name: category_id, Length: 100, dtype: object


,op_id,category_id,category_desc,desc_short,icd10_pcs,orin_time,orout_time,op_duration,LOS
1,446270725,0WJ,Anatomical Regions,Inspection of Peritoneal Cavity,0WJG0,1340,1550,210,48
7,466411896,0NQ,Head and Facial Bones,Repair Maxilla,0NQR0,2140,2230,90,2
10,430539801,09Q,Ear,Repair Nasal Septum,09QM0,1910,2050,140,2
11,466389608,09Q,Ear,Repair Right Tympanic Membrane,09Q70,1910,2000,90,2
12,487807079,09Q,Ear,Repair Right Tympanic Membrane,09Q70,76065,76125,60,23
...,...,...,...,...,...,...,...,...,...
128024,454166011,0WB,Anatomical Regions,Excision of Abdominal Wall,0WBFX,313220,313310,90,5
128025,431396325,0DN,Gastrointestinal System,Release Large Intestine,0DNE4,10940,11015,75,8
128026,449124488,0DN,Gastrointestinal System,Release Large Intestine,0DNE0,5000380,5000570,190,8
128029,419787421,0DB,Gastrointestinal System,Excision of Stomach,0DB64,13775,13950,175,9


## Visualize LOS Distributions for surgeries

In [15]:
import plotly.graph_objects as go

import pandas as pd

df = top_LOSvariance_df

fig = go.Figure()

pcs_list = top_LOSvariance_df['category_id']

for pcs in pcs_list:
    fig.add_trace(go.Violin(x=df['category_id'][df['category_id'] == pcs],
                            y=df['LOS'][df['category_id'] == pcs],
                            name=pcs,
                            box_visible=True,
                            meanline_visible=True))

fig.show()